## Singular Value Decomposition with implicit feedback (SVD++)

В работе Factorization Meets the Neighborhood описывается новая модификация SVD модели. Но для начала поговорим про явный и неявный отклик от пользователя (explicit and implicit user feedback). Если по итогу взаимодействия между пользователем и товаром мы знаем оценку $r_{ui}$, то это считается явным откликом. В противном случае мы знаем лишь об их взаимодействии и это неявный. Авторы статьи ассоциируют каждого пользователя с двумя группами товаров: $R(u)$ — множество товаров с явным откликом (рейтинги которых мы знаем) и $N(u)$ — множество товаров с неявным откликом (знаем лишь о наличии взаимодействия).


Метод SVD++ использует неявный отклик и в результате модель выглядит следующим образом:


$\hat{r}_{ui} =\mu + b_u + b_i + q^T_i \left( p_u + |N(u)|^{-1/2} \sum_{j \in N(u)} y_j \right) .$


В этом случае параметрами обучения будут:


$\Theta = \{ \mu, b_u, b_i, p_u, q_i, y_i| u \in U, i \in I\} .$

Поскольку неявный отклик иногда бывает недоступным, множество $N(u)$ можно заменить на $R(u)$, т.к. всегда выполняется $R(u) \subset N(u)$. А добавку данного метода можно расценивать как рекомендации товара к товару (item-item recommendation).

In [103]:
def svdpp():
    for epoch in range(epochs):
        for user in range(num_users):
            for item in range(num_items):
                # activation
                p_i = p[user, :]
                q_j = q[:,item]
                y_j = y[:, item]
                
                user_item_list = [i for i, e in enumerate(r[user,:]) if e != 0] # R_u (item indices which a user interacted with)
                sum_y_j = np.sum(y[:,user_item_list],axis=1) # to make sum of y_i
                r_hat[user,item] = bu[user, item] + bi[user, item] + mu[user, item] + \
                                   np.dot(np.transpose(q_j), p[user, :] + pow(len(user_item_list), -0.5)*sum_y_j)

                # descent
                e = r[user, item] - r_hat[user, item]

                p[user]   += 2*alpha*(e*q_j - lambda_1*p_i)
                q[:,item] += 2*alpha*(e*(p_i + pow(len(user_item_list), -0.5)*sum_y_j) - lambda_1*q_j)
                y[:, item]+= 2*alpha*(e*(pow(len(user_item_list), -0.5)*q_j - lambda_1*y_j))

                bu[user, item] += 2*beta*(e - lambda_2*bu[user, item])
                bi[user, item] += 2*beta*(e - lambda_2*bi[user, item])
                mu[user, item] += 2*beta*(e - lambda_2*mu[user, item])
            
    return r_hat

In [107]:
import numpy as np

num_users = 3
num_items = 5

# hyperparameters
latent_dim = 2
alpha = 0.01
beta = 0.01

# regularization
lambda_1 = 0.001
lambda_2 = 0.001

epochs = 1000

prob = 0.4
r = np.random.binomial(1, 1 - prob,(num_users, num_items))
r_hat = np.zeros([num_users,num_items])
print(r)

p = np.random.randn(num_users,latent_dim)
q = np.random.randn(latent_dim,num_items)
y = np.random.randn(latent_dim,num_items)

# Bias matrices
bu = np.random.randn(1,num_users).repeat(num_items, axis=0).transpose()
bi = np.random.randn(1,num_items).repeat(num_users, axis=0)
mu = np.random.randn(num_users,num_items)

# Gradient descent 
r_hat = svdpp()

print(r_hat)

[[1 1 1 1 1]
 [0 0 0 1 0]
 [1 1 0 0 1]]
[[ 9.99937088e-01  9.99549499e-01  9.99197410e-01  9.98193918e-01
   1.00017961e+00]
 [ 1.03833675e-04  1.25594326e-04  1.14350364e-04  1.00002456e+00
   3.23635132e-04]
 [ 1.00002283e+00  9.99593595e-01 -5.03802444e-04 -1.60011119e-03
   1.00027827e+00]]
